In [2]:
from info_nas.datasets.networks.pretrained import pretrain_network_cifar
from info_nas.datasets.networks.utils import load_nasbench
from nasbench_pytorch.datasets.cifar10 import prepare_dataset

In [3]:
seed = 42
batch_size = 32

dataset = prepare_dataset(batch_size, root='../data/', validation_size=1000, random_state=seed)
train, n_train, val, n_val, test, n_test = dataset

--- Preparing CIFAR10 Data ---
Files already downloaded and verified
Files already downloaded and verified
--- CIFAR10 Data Prepared ---


In [6]:
print('train: ', n_train)
print('validation: ', n_val)
print('test: ', n_test)

train:  49000
validation:  1000
test:  10000


In [7]:
nasbench_path = '/home/gabi/diplomka/nasbench/nasbench_only108.tfrecord'

nb = load_nasbench(nasbench_path)

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 37 seconds


In [9]:
nets = nb[:50]

In [10]:
nets[0]

('00005c142e6f48ac74fdcf73e3439874',
 ['input',
  'conv3x3-bn-relu',
  'maxpool3x3',
  'conv3x3-bn-relu',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'output'],
 array([[0, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8))

In [22]:
from nasbench_pytorch.model import Network

num_labels = 10

def to_model(ops, mat):
    return Network((mat, ops), num_labels)

net_dict = {v[0]: to_model(v[1], v[2]) for v in nets}

In [24]:
from info_nas.datasets.networks.utils import save_trained_net

for net_hash, net in net_dict.items():
    net, metrics = pretrain_network_cifar(net, train, val, test, num_tests=n_test, num_epochs=108)
    
    save_trained_net(net_hash, net, info=metrics, net_args=[num_labels])

/home/gabi/anaconda3/envs/pyt/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


TypeError: 'DataLoader' object is not callable